In [1]:
import os
import functions as fun
from geopy import distance
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import pandas as pd

In [2]:
sub_SE = pd.read_csv("../Data/sub_SE.csv")


# Seattle


In [3]:
Seattle = 47.6251419,-122.3268577

In [4]:
load_dotenv()

True

In [5]:
token_foursquare = os.getenv("Token_foursquare")

- First we look for Starbucks close to the office and clean the results keeping only the columns that we are interested about 

In [6]:
starbucks_SE = fun.get_results("starbucks", Seattle, 10, 1000, token_foursquare)
df_SE = fun.clean_results_first(starbucks_SE, "Coffee")

- We look for preschools or kindergarten around the office and clean the results keeping only the columns that we are interested about

In [7]:
kinder_SE = fun.get_results("preschool", Seattle, 10, 3000, token_foursquare)
df_SE = fun.clean_results(kinder_SE, df_SE, "Preschool")

- We look for nightclubs around the office and clean the results keeping only the columns that we are interested about


In [8]:
club_SE = fun.get_results("night club", Seattle, 10, 3000, token_foursquare)
df_SE = fun.clean_results(club_SE, df_SE, "Club")

- We look for vegan restaurants around the office and clean the results keeping only the columns that we are interested about

In [9]:
vegan_SE = fun.get_results("vegan", Seattle, 10, 3000, token_foursquare)
df_SE = fun.clean_results(vegan_SE, df_SE, "Vegan restaurant")


- We look for basketball arena. However, there are not major teams playing nowadays in Seattle. We pray for the return of the Supersonics


- We look for dog hairdressers restaurants around the office and clean the results keeping only the columns that we are interested about

In [10]:
pet_SE = fun.get_results("pet grooming", Seattle, 5, 3000, token_foursquare)
df_SE = fun.clean_results(pet_SE, df_SE, "Pet hairdresser")

In [11]:
Basketball_SE = fun.get_results("Climate Pledge Arena", Seattle, 1, 5000, token_foursquare)
df_SE = fun.clean_results(Basketball_SE, df_SE, "Basketball arena")

- We look for the closest Airport to office and clean the results keeping only the columns that we are interested about

In [12]:
Airports_SE = fun.get_results_airport("seattle tacoma airport", Seattle, 1, 50000, token_foursquare)
df_SE = fun.clean_results_lo(Airports_SE, df_SE, "Airport")

- We add the data of the new office that we have selected

In [13]:
df_SE = fun.add_company(df_SE, "South Lake Union, Seattle, Washington 98109", "South Lake Union", 47.624730, -122.335911)

- We create a new row with the distance from the office to the diferent places

In [14]:
df_SE["Distance"] = df_SE.apply(fun.calc_distance_SE, axis = 1).round(2)

In [15]:
df_SE

,name,address,neighborhood,lat,lon,category,Distance
0,Company,"South Lake Union, Seattle, Washington 98109",South Lake Union,47.624730,-122.335911,NaN,682.03
1,Starbucks,"515 Westlake Ave N (Mercer), Seattle, WA 98109",Belltown,47.624391,-122.338900,Coffee,909.00
2,Starbucks,"101 Broadway E (Denny), Seattle, WA 98102",Broadway,47.623015,-122.320684,Coffee,520.83
3,Starbucks,"1730 Minor Ave, Seattle, WA 98101",Belltown,47.616933,-122.329743,Coffee,938.11
4,Starbucks,"434 Broadway E (at E Republican St), Seattle, ...",Broadway,47.623033,-122.320688,Coffee,519.66
5,Starbucks,"1600 E Olive Way, Seattle, WA 98102",Broadway,47.619484,-122.325060,Coffee,643.41
6,Starbucks,"442 Terry Ave N, Seattle, WA 98109",Belltown,47.622870,-122.336875,Coffee,794.20
7,Starbucks,"101 Broadway E (Denny), Seattle, WA 98102",Broadway,47.623015,-122.320684,Coffee,520.83
8,Capitol Hill Cooperative Preschool,"1200 10th Ave E (at East Highland Dr), Seattle...",Broadway,47.630635,-122.319880,Preschool,805.01
9,KinderCare Learning Center,"1827 8th Ave, Seattle, WA 98101",Belltown,47.614661,-122.335244,Preschool,1324.89


In [16]:
SE_map = Map(location = Seattle, zoom_start = 12)

In [17]:
for index, row in df_SE.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}
    
    #2. Icon
    if row["category"] == "Coffee":        
        icon = Icon (
            color="white",
            opacity = 0.6,
            prefix = "fa",
            icon="coffee",
            icon_color = "black"
        )
    elif row["category"] == "Preschool":
        icon = Icon (
            color="pink",
            opacity = 0.6,
            prefix = "fa",
            icon="graduation-cap",
            icon_color = "yellow"
        )
    elif row["category"] == "Club":
        icon = Icon (
            color="darkpurple",
            opacity = 0.6,
            prefix = "fa",
            icon="glass",
            icon_color = "white"
        )
    elif row["category"] == "Vegan restaurant":
        icon = Icon (
            color="green",
            opacity = 0.6,
            prefix = "fa",
            icon="cutlery",
            icon_color = "white"
        )
    elif row["category"] == "Basketball arena":
        icon = Icon (
            color="orange",
            opacity = 0.6,
            prefix = "fa",
            icon="futbol-o",
            icon_color = "white"
        )
    elif row["category"] == "Airport":
        icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="plane",
            icon_color = "white"
            
        )
    elif row["category"] == "Pet hairdresser":
        icon = Icon (
            color="red",
            opacity = 0.6,
            prefix = "fa",
            icon="scissors",
            icon_color = "white"
        )
    else:
        row["category"] == "Company",
        icon = Icon (
        color="black",
        opacity = 0.9,
        prefix = "fa",
        icon = "briefcase",
        icon_color = "white",
        icon_size=(30, 30)
        )
            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(SE_map)

In [18]:
for index, row in sub_SE.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["Lat"], row["Long"]], "tooltip": row["name"]}
    
    #2. Icon       
    icon = Icon (
    color="white",
    opacity = 0.6,
    prefix = "fa",
    icon="building",
    icon_color = "black"
        )

            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(SE_map)

In [19]:
SE_map

In [20]:
SE_map.save('../images/Seattle_map.html')